In [1]:
import sys
import importlib
import pickle

# data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# data analysis
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# pytorch specific
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

# project specific
sys.path.append('../')
from src import config
from src.utils import visualisation, benchmark, helpers, monitoring, experiment
from src.models import model


from matplotlib.animation import FuncAnimation
from IPython.display import HTML

pd.options.display.width = 1000
DEVICE = torch.device(config["DEVICE"])

In [23]:
importlib.reload(model)
#with open('../workfiles/BRCA_ds.pkl', 'rb') as f:
data_param = '../workfiles/light_BRCA_ds.pkl'
config = {
    "dropout" : 0.5,
    "latent_dim" : 16,
    "convolution" : True,
    "transformer" : False,
    "num_layers" :  2,
    "n_epoch" : 12
}

e = experiment.Experiment(data_param=data_param, model_param=config, )
print(e.model)
e.run()


/opt/homebrew/anaconda3/envs/thesis/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


input shape : 3242
(1231, 3242)
(1231, 1, 3242)
train data: 1231
Autoencoder(
  (encoder): Sequential(
    (0): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
    (1): LeakyReLU(negative_slope=0.05)
    (2): Dropout(p=0.5, inplace=False)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
    (5): LeakyReLU(negative_slope=0.05)
    (6): Dropout(p=0.5, inplace=False)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): LazyLinear(in_features=0, out_features=16, bias=True)
    (10): LeakyReLU(negative_slope=0.05)
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=12928, bias=True)
    (1): Unflatten(dim=1, unflattened_size=(64, 202))
    (2): Upsample(scale_factor=2.0, mode='nearest')
    (3): ConvTranspose1d(64, 64, kernel_size=(7,), stride=(2,), padding=(3,))
    (4): Lea

 33%|███▎      | 4/12 [00:23<00:47,  5.91s/it]


KeyboardInterrupt: 

In [20]:
# let's get a sample a run it layer by layer
x = iter(e.dataloader).__next__()

In [21]:
e.model(x)

tensor([[[0.4993, 0.5164, 0.4786,  ..., 0.5219, 0.5181, 0.4883]],

        [[0.4931, 0.5052, 0.4760,  ..., 0.5269, 0.5270, 0.4883]],

        [[0.4890, 0.4993, 0.4832,  ..., 0.5509, 0.5219, 0.4888]],

        ...,

        [[0.5203, 0.5016, 0.4753,  ..., 0.5363, 0.5191, 0.5015]],

        [[0.5056, 0.4759, 0.4818,  ..., 0.5378, 0.5149, 0.4852]],

        [[0.5227, 0.4886, 0.4835,  ..., 0.5479, 0.5263, 0.4839]]],
       grad_fn=<SigmoidBackward0>)

In [16]:
x_ = x
for layer in e.model.encoder:
    print(layer)
    x_ = layer(x_)
print(x_.shape)

Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
LeakyReLU(negative_slope=0.05)
Dropout(p=0.5, inplace=False)
MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
LeakyReLU(negative_slope=0.05)
Dropout(p=0.5, inplace=False)
MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv1d(64, 128, kernel_size=(7,), stride=(2,), padding=(3,))
LeakyReLU(negative_slope=0.05)
Dropout(p=0.5, inplace=False)
MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv1d(128, 256, kernel_size=(7,), stride=(2,), padding=(3,))
LeakyReLU(negative_slope=0.05)
Dropout(p=0.5, inplace=False)
MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv1d(256, 512, kernel_size=(7,), stride=(2,), padding=(3,))
LeakyReLU(negative_slope=0.05)
Dropout(p=0.5, inplace=False)
MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Flatten(start_di

In [17]:
for layer in e.model.decoder:
    print(layer)
    x_ = layer(x_)
    print(x_.shape)

Linear(in_features=16, out_features=384, bias=True)
torch.Size([32, 384])
Unflatten(dim=1, unflattened_size=(128, 3))
torch.Size([32, 128, 3])
Upsample(scale_factor=2.0, mode='nearest')
torch.Size([32, 128, 6])
ConvTranspose1d(128, 512, kernel_size=(7,), stride=(2,), padding=(3,))
torch.Size([32, 512, 11])
LeakyReLU(negative_slope=0.05)
torch.Size([32, 512, 11])
Dropout(p=0.5, inplace=False)
torch.Size([32, 512, 11])
Upsample(scale_factor=2.0, mode='nearest')
torch.Size([32, 512, 22])
ConvTranspose1d(512, 256, kernel_size=(7,), stride=(2,), padding=(3,))
torch.Size([32, 256, 43])
LeakyReLU(negative_slope=0.05)
torch.Size([32, 256, 43])
Dropout(p=0.5, inplace=False)
torch.Size([32, 256, 43])
Upsample(scale_factor=2.0, mode='nearest')
torch.Size([32, 256, 86])
ConvTranspose1d(256, 128, kernel_size=(7,), stride=(2,), padding=(3,))
torch.Size([32, 128, 171])
LeakyReLU(negative_slope=0.05)
torch.Size([32, 128, 171])
Dropout(p=0.5, inplace=False)
torch.Size([32, 128, 171])
Upsample(scale_fac

In [18]:
i = 7
32 * (2 ** i)

4096

In [ ]:
e.run()
